In [1]:
import mnist
import numpy as np
from tqdm.auto import tqdm

W - [W_L of shape (num_inputs, num_neurons)] of length l

In [2]:
train_images = mnist.train_images()
imean = np.mean(train_images, axis=0)
istd = np.std(train_images, axis=0)
istd[istd == 0] = 1
train_images = (train_images - imean) / istd
# train_images = train_images / 255

train_labels = mnist.train_labels()

In [3]:
train_images.shape

(60000, 28, 28)

In [3]:
N1 = 128
N2 = 10
W = [np.random.uniform(-1, 1, size=(28**2 + 1, N1)), np.random.uniform(-1, 1, size=(N1 + 1, N2))]

In [4]:
def forward(x):
    y = np.array([1] + x.ravel().tolist())
    out = []
    for l in range(2):
        v = W[l].T @ y
        y = np.tanh(v)
        out.append(y)
        y = np.array([1] + y.tolist())
    return out

In [5]:
train_images[5]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -4.41807799e-03, -5.75481961e-03, -4.08251693e-03,
        -4.08251693e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -4.08251693e-03, -4.70968827e-03,
        -8.79934640e-03, -1.15905591e-02, -1.47589798e-02,
        -1.92848546e-02, -2.46717975e-02, -2.90710271e-02,
        -3.05926583e-02, -3.11640127e-02, -3.19628719e-02,
        -3.02025355e-02, -3.13102175e-02, -2.83833960e-02,
        -2.31191906e-02, -1.91666260e-02, -1.67723008e-02,
        -1.09963601e-02, -8.32

In [5]:
forward(train_images[5])

[(array([ 6.84521644e+00,  1.19526784e+00,  8.44821642e+00,  8.79782006e+00,
          6.04318636e+00,  1.13083928e+00, -1.36762854e+00, -5.11667261e+00,
         -2.21812036e+00,  1.47407418e+00, -1.00432979e+01,  1.39282798e+01,
          8.87306845e+00, -2.01030572e+01,  1.30106431e+01, -1.16765459e+01,
          1.27578045e+01, -2.33800095e+00,  1.79763765e+00,  1.28318147e+01,
         -1.67921243e+01, -4.05705495e+00, -8.33882141e+00, -1.55573095e+01,
         -7.78545779e+00,  1.32598275e+01,  5.66708920e+00, -1.73562826e+01,
         -1.45791508e+00,  1.44883759e+01, -5.16720657e+00,  1.32446012e+01,
          1.23979593e+00, -2.11370782e+00,  1.33690671e+01, -7.19755869e-01,
         -3.88485286e+00, -7.37500233e+00,  2.25167827e+00,  2.48113947e+00,
         -1.60905683e+01, -7.54843154e+00, -2.43450461e+00, -4.27283562e+00,
         -2.10272183e+01, -1.02279165e+01, -1.57005714e+01, -6.89679514e-01,
         -2.39292996e+00,  7.69262258e+00,  7.33408125e+00, -5.31849114e+00,

In [5]:
def evaluate():
    correct = 0
    loss = []
    for x, l in tqdm(zip(train_images, train_labels), total=train_images.shape[0]):
        f = forward(x)
        y = f[1]
        d = np.zeros(10)
        d[l] = 1
        loss.append((d - y) ** 2)
        y = np.argmax(y)
        if y == l:
            correct += 1
    return np.mean(loss), correct * 100 / train_images.shape[0]

In [6]:
evaluate()

  0%|          | 0/60000 [00:00<?, ?it/s]

(0.957897370623737, 11.678333333333333)

In [6]:
def epoch(eta=0.001):
    np.random.shuffle(train_images)
    for x, l in tqdm(zip(train_images, train_labels), total=train_images.shape[0]):
        f = forward(x)
        d = np.zeros(10)
        d[l] = 1
        delta_2 = (d - f[1]) * (1 - f[1] ** 2)
        delta_1 = (1 - f[0] ** 2) * (W[1] @ delta_2)
        del_W_1 = eta * np.outer(np.array([1] + x.ravel().tolist()), delta_1)
        del_W_2 = eta * np.outer(np.array([1] + f[0].tolist()), delta_2)
        W[0] += del_W_1
        W[1] += del_W_2
    return

In [7]:
epoch(0.00001)
evaluate()

  0%|          | 0/60000 [00:00<?, ?it/s]

  0%|          | 0/60000 [00:00<?, ?it/s]

(0.08997633324219917, 10.441666666666666)

In [7]:
for _ in range(5):
    epoch(0.1)
    print(evaluate())

  0%|          | 0/60000 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (128,) (129,) 